# 1. Pragmatic Ambiguous Sentences Dataset
- Extraction information of each Utterance

## 2) Extract emotion labeles from json

In [43]:
import os
import json
import pandas as pd

In [103]:
def parse_json_objects(unit_folder, json_path):
    """
    1) JSON 파일 로드
    2) data[frame_no][object_id] 순회
    3) object_id in [1, 2], label='person', text 필드 존재 시
       - frame, predicate, emotion, label, text 내 script, script_start, script_end, intent, strategy, morpheme
       - person_id
       - emotion 내 각 (valence, emotion, arousal) for image, sound, multimodal, text
    4) 결과를 (dict) 리스트 형태로 반환
    """

    records = []
    seen_text_triples = set()  # (script, script_start, script_end) 중복 체크용 집합

    if not os.path.isfile(json_path):
        print(f"[스킵] 파일 없음: {json_path}")
        return records

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except UnicodeDecodeError as e:
        print(f"[에러] UTF-8로 디코딩할 수 없는 파일입니다: {json_path} - {str(e)}")
        return records

    # data 필드 (프레임별 객체)
    frame_dict = data.get("data", {})
    n_record = 0
    for frame_no, obj_dict in frame_dict.items():
        # obj_dict: 각 object_id -> object_info
        for object_id, info in obj_dict.items():
            # object_id가 '1' 또는 '2'만 추출 (문자열이므로 int 변환 후 비교 가능)
            try:
                obj_id_int = int(object_id)
            except:
                continue

            if obj_id_int not in [1, 2]:
                continue

            # label이 "person"인지 확인
            label = info.get("label", "")  # .lower()
            if label != "사람" and label != "person":
                continue

            # text 필드가 존재하는지 확인
            text_info = info.get("text", None)
            if not text_info:
                # text가 없으면 스킵
                continue
            
            # text 필드
            script = text_info.get("script", None)
            script_start = text_info.get("script_start", None) # 시작 프레임 
            script_end = text_info.get("script_end", None)  # 종료 프레임             
            
            # (script, script_start, script_end) 중복 확인
            text_key = (script, script_start, script_end)
            if text_key in seen_text_triples:
                # 이미 동일한 텍스트 구간이 기록되었다면 스킵
                continue
            seen_text_triples.add(text_key)
            n_record += 1

            intent = text_info.get("intent", None)
            strategy = text_info.get("strategy", None)
            morpheme = text_info.get("morpheme", None)  # 텍스트 인코더 입력 고려하여 남김 
            
            # data 객체 층위, 필요한 필드 추출
            clip_id = data.get("clip_id", None)
            actor = data.get("actor", None)
            situation = data.get("situation", None)
            category = data.get("category", None)
            nr_frame = data.get("nr_frame", None)  # 전체 프레임 수
            predicate = info.get("predicate", {})
            emotion = info.get("emotion", {})
            person_id = info.get("person_id", None)
            # emotion 
            # (image/sound/multimodal/text 중 image 제외한 각각의 valence, emotion, arousal)
            sound_emo = emotion.get("sound", {})
            multi_emo = emotion.get("multimodal", {})
            text_emo = emotion.get("text", {})
            # (emotion, valence, arousal) 추출
            sound_emotion = sound_emo.get("emotion", None)
            sound_valence = sound_emo.get("valence", None)
            sound_arousal = sound_emo.get("arousal", None)
            multimodal_emotion = multi_emo.get("emotion", None)
            multimodal_valence = multi_emo.get("valence", None)
            multimodal_arousal = multi_emo.get("arousal", None)
            text_emotion = text_emo.get("emotion", None)
            text_valence = text_emo.get("valence", None)
            text_arousal = text_emo.get("arousal", None)

            # record 생성
            file_index = f'{unit_folder}_{clip_id}_{n_record}'
            record = {
                "file_index": file_index,
                "clip_id": clip_id,
                "actor": actor, 
                "category": category,
                "nr_frame": nr_frame,                  
                "frame": frame_no,
                "object_id": object_id,
                "label": label,
                "predicate": predicate,   # dict로 저장
                "person_id": person_id,
                # text info
                "script": script,
                "script_start": script_start,
                "script_end": script_end,
                "intent": intent,
                "strategy": strategy,
                "morpheme": morpheme,
                # emotion - sound
                "sound_valence": sound_valence,
                "sound_emotion": sound_emotion,
                "sound_arousal": sound_arousal,
                # emotion - multimodal
                "multimodal_valence": multimodal_valence,
                "multimodal_emotion": multimodal_emotion,
                "multimodal_arousal": multimodal_arousal,
                # emotion - text
                "text_valence": text_valence,
                "text_emotion": text_emotion,
                "text_arousal": text_arousal
            }
            records.append(record)

    return records

In [135]:
def process_all_json(input_data_root_dir, output_path, list_unit_folder, clip_range):
    all_dfs = []
    for i, unit_name in enumerate(list_unit_folder):
        for clip_id in range(clip_range[i][0], clip_range[i][1]):  # for clip_id in range(1, 401):
            clip_folder = f"clip_{clip_id}"
            input_path = os.path.join(input_data_root_dir, unit_name, clip_folder)
            if not os.path.isdir(input_path):
                print(f"[스킵] 폴더 없음: {input_path}")
                continue
            json_path = os.path.join(input_path, f"clip_{clip_id}.json")   
            if not os.path.isfile(json_path):
                print(f"[스킵] JSON 파일 없음: {json_path}")
                continue
            data_records = parse_json_objects(unit_name, json_path)
            if not data_records:
                print(f"[완료] 발화 구간 없음: clip_{clip_id}")
                continue
            df = pd.DataFrame(data_records)
            df["unit_folder"] = unit_name
            print(f"[INFO] clip_{clip_id} -> 총 {len(df)}개 발화 구간 레코드 dataframe이 저장되었습니다.")
            all_dfs.append(df)  # 400개 clip의 df 저장  
        df_all_data = pd.concat(all_dfs, ignore_index=True)
        unit_dfs = df_all_data[df_all_data["unit_folder"] == unit_name].copy()
        save_path = os.path.join(output_path, f'{unit_name}.csv')
        unit_dfs.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"[INFO] 400개 clip의 CSV 파일이 저장되었습니다: {save_path}")
    
    if not all_dfs:
        print("[INFO] 처리할 데이터가 없습니다.")
        return None
    df_all_data = pd.concat(all_dfs, ignore_index=True)
    merged_filename = "merged_data.csv"
    save_path = os.path.join(output_path, merged_filename)
    df_all_data.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"[INFO] 모든 DataFrame이 합쳐진 CSV 파일이 저장되었습니다: {save_path}")
    
    return df_all_data

In [137]:
input_data_root_dir = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/멀티모달 영상'
output_path = 'G:/내 드라이브/aiffel/aiffelthon/multimodal/labled_data'
list_unit_folder = ['0001-0400', '0401-0800', '0801-1200', '1201-1600',\
                   '1601-2000', '2001-2400', '2401-2800', '2801-3200', '3201-3600', '3601-4000',\
                   '4001-4400', '4401-4800', '4801-5200', '5201-5600']
clip_range = [
    (1, 401),
    (401, 801),
    (801, 1201),
    (1201, 1601),
    (1601, 2001),
    (2001, 2401),
    (2401, 2801),
    (2801, 3201),
    (3201, 3601),
    (3601, 4001),
    (4001, 4401),
    (4401, 4801),
    (4801, 5201),
    (5201, 5601)
]


In [ ]:
# process_all_json(input_data_root_dir, output_path, list_unit_folder, clip_range)